In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool

In [ ]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [ ]:
df.head()

In [ ]:
coupon_list=df.coupon_id.unique()

In [ ]:
def get_similar_coupon_mean(c):
    coups=coupon[coupon.item_id.isin(coupon[coupon.coupon_id==c].item_id)].coupon_id.unique()
    coups=coups[np.where(coups!=c)[0]]
    return c,train[train.coupon_id.isin(coups)].redemption_status.sum(),train[train.coupon_id.isin(coups)].redemption_status.mean()

In [ ]:
%%time
cols=['coupon_id','tot_redemption','mean_redemption']
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_similar_coupon_mean, [coup for coup in coupon_list])
finally: 
    pool.close()
    pool.join()

In [ ]:
pd.DataFrame(data_outputs,columns=cols).to_csv('../coupon_mean_encoding_v2.csv',index=False)

In [ ]:
transaction=transaction.merge(item,on='item_id')

In [ ]:
coupon=coupon.merge(item,on='item_id')

In [ ]:
coupon.head()

In [ ]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                         (transaction.brand.isin(coupon[coupon.coupon_id==data_['coupon_id']].brand))\
               &(transaction.date<data_['start_date'])]
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [ ]:
import datetime
temp=df[['customer_id','coupon_id','start_date','id','end_date']]
temp['cut_off_date']=pd.to_datetime(df.start_date) -datetime.timedelta(150)

In [ ]:
%%time
cols=['id','num_bought_brand_cust','num_discount_availed_brand_cust','frac_discount_availed_brand_cust','sum_discount_brand_cust',
'selling_price_sum_brand_cust',
'coupon_discount_mean_brand_cust','selling_price_mean_brand_cust','other_discount_sum_brand_cust',
      'other_discount_mean_brand_cust','cd_sp_ratio_mean_brand_cust','cd_sp_ratio_sum_brand_cust'
     ]

try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

In [ ]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [ ]:
feat_df.to_csv('../brand_cust_feats_no_lag_v2.csv',index=False)

In [ ]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.brand.isin(coupon[coupon.coupon_id==data_['coupon_id']].brand))\
               &(transaction.date<data_['start_date'])]

    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [ ]:
%%time
# cols=['id','num_bought_brand','num_discount_availed_brand','frac_discount_availed_brand','sum_discount_brand']
cols=['id','num_bought_brand','num_discount_availed_brand','frac_discount_availed_brand','sum_discount_brand',
'selling_price_sum_brand',
'coupon_discount_mean_brand','selling_price_mean_brand','other_discount_sum_brand',
      'other_discount_mean_brand','cd_sp_ratio_mean_brand','cd_sp_ratio_sum_brand'
     ]
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

In [ ]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
feat_df.to_csv('../brand_feats_no_lag_v2.csv',index=False)

In [ ]:
# a=pd.read_csv('../brand_cust_feats_no_lag_v2.csv')
# b=pd.read_csv('../../av_amex/brand_cust_feats_no_lag_v2.csv')
# (a.mean()==b.mean()).value_counts()